## Dependencies

In [1]:
import gc
import re
import warnings
from joblib import load
import tensorflow_hub as hub
from tensorflow.keras import Model, optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Concatenate, BatchNormalization, Activation, Average, Add, Reshape
from tensorflow.keras.layers import GlobalAveragePooling1D, Embedding, LSTM, Conv1D, SpatialDropout1D, Bidirectional, Flatten
from googleqa_utilityscript import *


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
module_url = '/kaggle/input/universalsentenceencodermodels/universal-sentence-encoder-models/use'
encoder_path = '/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'
model_path = '/kaggle/input/83-googleq-a-train-glove-lstm-10paragraphs/model.h5'
tokenizer_path = '/kaggle/input/83-googleq-a-train-glove-lstm-10paragraphs/tokenizer.joblib'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

for feature in text_features:
    test[feature + '_uncased'] = test[feature]

In [5]:
def split_paragraphs(row):
    splited_body = re.split('\n\n\n\n\n|\n\n\n\n|\n\n\n|\n\n', row['question_body_uncased'])
    splited_answer = re.split('\n\n\n\n\n|\n\n\n\n|\n\n\n|\n\n', row['answer_uncased'])
    for i in range(len(splited_body))[:10]:
        row['body_paragraph_%s' % (i+1)] = splited_body[i]
    for i in range(len(splited_answer))[:10]:
        row['answer_paragraph_%s' % (i+1)] = splited_answer[i]
    return row

test = test.apply(split_paragraphs, axis=1)

# Model parameters

In [6]:
EMBEDDDING_SIZE = 512
N_CLASS = len(target_cols)
MAX_FEATURES = 100000
TITLE_MAX_LEN = 30
BODY_MAX_LEN = 30
ANSWER_MAX_LEN = 30

## Test set

In [7]:
body_ft = ['body_paragraph_1', 'body_paragraph_2', 'body_paragraph_3', 'body_paragraph_4', 'body_paragraph_5', 
           'body_paragraph_6', 'body_paragraph_7', 'body_paragraph_8', 'body_paragraph_9', 'body_paragraph_10']
answer_ft = ['answer_paragraph_1', 'answer_paragraph_2', 'answer_paragraph_3', 'answer_paragraph_4', 'answer_paragraph_5', 
             'answer_paragraph_6', 'answer_paragraph_7', 'answer_paragraph_8', 'answer_paragraph_9', 'answer_paragraph_10']

tokenizer = load(tokenizer_path)


# Test features
# X_test_title = test['question_title']
# X_test_body = test['question_body']
# X_test_answer = test['answer']
X_test_title_seq = test['question_title_uncased']
X_test_body_fts = test[body_ft].astype(str)
X_test_answer_fts = test[answer_ft].astype(str)

# Tokenize the sentences
X_test_title_seq = tokenizer.texts_to_sequences(X_test_title_seq)
for ft in body_ft:
    X_test_body_fts[ft] = tokenizer.texts_to_sequences(X_test_body_fts[ft])
for ft in answer_ft:
    X_test_answer_fts[ft] = tokenizer.texts_to_sequences(X_test_answer_fts[ft])

X_test_body_seq = np.empty((X_test_body_fts.shape[0], BODY_MAX_LEN, X_test_body_fts.shape[1]))
X_test_answer_seq = np.empty((X_test_answer_fts.shape[0], ANSWER_MAX_LEN, X_test_answer_fts.shape[1]))

# Pad the sentences
X_test_title_seq = pad_sequences(X_test_title_seq, maxlen=TITLE_MAX_LEN)
for index, ft in enumerate(body_ft):
    X_test_body_seq[...,index] = pad_sequences(X_test_body_fts[ft], maxlen=BODY_MAX_LEN)
for index, ft in enumerate(answer_ft):
    X_test_answer_seq[...,index] = pad_sequences(X_test_answer_fts[ft], maxlen=ANSWER_MAX_LEN)
    
X_test = [X_test_title_seq, X_test_body_seq, X_test_answer_seq]

# Model

In [8]:
# use_embed = hub.load(module_url)

# def USEEmbedding(x):
#     return use_embed(tf.squeeze(tf.cast(x, tf.string)))

# def encoder_block(input_layer):
#     encoder = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_layer)
    
#     return encoder

def encoder_block_parag(input_layer, index, name_pre):
    name = '%s_parag_%d' % (name_pre, index)
    encoder = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False, name='enc_'+name)(input_layer[...,index])
    encoder = SpatialDropout1D(0.2)(encoder)
    encoder = LSTM(64, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=True)(encoder)

    return encoder

## Load embedding

In [9]:
glove_embedding_matrix, glove_unknown_words = build_matrix(tokenizer.word_index, encoder_path, MAX_FEATURES)
embedding_name = 'GloVe'
print('n unknown words "%s": %d' % (embedding_name, len(glove_unknown_words)))
print('"%s" embedding shape: %s' % (embedding_name, glove_embedding_matrix.shape))

n unknown words "GloVe": 19913
"GloVe" embedding shape: (58007, 300)


In [10]:
# Title sequence
input_title_seq = Input(shape=(TITLE_MAX_LEN,), dtype=tf.float32, name='input_title_seq')
title = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_title_seq)
title = SpatialDropout1D(0.2)(title)
title = LSTM(128, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=True)(title)
title_out = GlobalAveragePooling1D()(title)

# Body sequence
input_body_seq = Input(shape=(*X_test_body_seq.shape[1:], ), dtype=tf.float32, name='input_body_seq')

body1_out = encoder_block_parag(input_body_seq, 0, 'body')
body2_out = encoder_block_parag(input_body_seq, 1, 'body')
body3_out = encoder_block_parag(input_body_seq, 2, 'body')
body4_out = encoder_block_parag(input_body_seq, 3, 'body')
body5_out = encoder_block_parag(input_body_seq, 4, 'body')
body6_out = encoder_block_parag(input_body_seq, 5, 'body')
body7_out = encoder_block_parag(input_body_seq, 6, 'body')
body8_out = encoder_block_parag(input_body_seq, 7, 'body')
body9_out = encoder_block_parag(input_body_seq, 8, 'body')
body10_out = encoder_block_parag(input_body_seq, 9, 'body')

body_out = Concatenate()([body1_out, body2_out, body3_out, body4_out, body5_out, body6_out, body7_out, body8_out, body9_out, body10_out])
body_out = SpatialDropout1D(0.5)(body_out)
body_out = LSTM(256, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=False)(body_out)

# Answer sequence
input_answer_seq = Input(shape=(*X_test_body_seq.shape[1:], ), dtype=tf.float32, name='input_answer_seq')

answer1_out = encoder_block_parag(input_answer_seq, 0, 'answer')
answer2_out = encoder_block_parag(input_answer_seq, 1, 'answer')
answer3_out = encoder_block_parag(input_answer_seq, 2, 'answer')
answer4_out = encoder_block_parag(input_answer_seq, 3, 'answer')
answer5_out = encoder_block_parag(input_answer_seq, 4, 'answer')
answer6_out = encoder_block_parag(input_answer_seq, 5, 'answer')
answer7_out = encoder_block_parag(input_answer_seq, 6, 'answer')
answer8_out = encoder_block_parag(input_answer_seq, 7, 'answer')
answer9_out = encoder_block_parag(input_answer_seq, 8, 'answer')
answer10_out = encoder_block_parag(input_answer_seq, 9, 'answer')

answer_out = Concatenate()([answer1_out, answer2_out, answer3_out, answer4_out, answer5_out, answer6_out, answer7_out, answer8_out, answer9_out, answer10_out])
answer_out = SpatialDropout1D(0.5)(answer_out)
answer_out = LSTM(256, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=False)(answer_out)


# Output
x = Concatenate()([title_out, body_out, answer_out])
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', kernel_initializer='lecun_normal')(x)
x = Dropout(0.5)(x)

output = Dense(N_CLASS, activation='sigmoid', kernel_initializer='lecun_normal', name='output')(x)
model = Model(inputs=[input_title_seq, input_body_seq, input_answer_seq], outputs=[output])

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_body_seq (InputLayer)     [(None, 30, 10)]     0                                            
__________________________________________________________________________________________________
input_answer_seq (InputLayer)   [(None, 30, 10)]     0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 30)]         0           input_body_seq[0][0]             
__________________________________________________________________________________________________
tf_op_layer_strided_slice_1 (Te [(None, 30)]         0           input_body_seq[0][0]             
______________________________________________________________________________________________

# Make predictions

In [11]:
Y_test = model.predict(X_test)

In [12]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.889843,0.556504,0.190691,0.502035,0.648114,0.594337,0.616048,0.574688,0.433824,...,0.848521,0.898270,0.614329,0.953633,0.960683,0.803494,0.125436,0.082796,0.856508,0.917889
1,46,0.868259,0.529471,0.007667,0.648475,0.740400,0.844657,0.544909,0.459251,0.164106,...,0.724630,0.924199,0.622466,0.957496,0.972080,0.825586,0.861088,0.202048,0.177402,0.879927
2,70,0.930418,0.693056,0.078311,0.751628,0.881389,0.851312,0.628066,0.594645,0.405350,...,0.879821,0.908195,0.634800,0.953112,0.961718,0.825872,0.039545,0.030916,0.788602,0.914151
3,132,0.887192,0.489998,0.001246,0.827434,0.825437,0.969742,0.534448,0.401864,0.202024,...,0.742014,0.976073,0.702225,0.989369,0.993620,0.921566,0.818967,0.135827,0.797869,0.934518
4,200,0.885956,0.507934,0.031820,0.771479,0.859799,0.862795,0.585876,0.512806,0.383510,...,0.780148,0.931297,0.657081,0.964786,0.969435,0.852762,0.172329,0.095412,0.739864,0.919963


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.876585,0.582256,0.041251,0.700811,0.791369,0.840107,0.569911,0.491033,0.238761,...,0.775435,0.921906,0.642222,0.959023,0.967026,0.845788,0.535565,0.149911,0.511300,0.896396
std,2812.670060,0.049298,0.124953,0.075415,0.106130,0.103225,0.133526,0.042896,0.072648,0.129755,...,0.090571,0.030661,0.038725,0.018230,0.018572,0.041780,0.324586,0.075506,0.277528,0.026931
min,39.000000,0.751670,0.363886,0.000255,0.345954,0.252240,0.219164,0.494957,0.363958,0.018492,...,0.542399,0.820410,0.548143,0.887656,0.896001,0.718817,0.002368,0.004686,0.027674,0.817108
25%,2572.000000,0.837489,0.469214,0.003937,0.645417,0.729983,0.808347,0.537014,0.438112,0.131684,...,0.704980,0.903910,0.617107,0.948811,0.956353,0.818291,0.218706,0.096170,0.265194,0.878978
50%,5093.000000,0.884454,0.570478,0.009741,0.707598,0.804178,0.886086,0.564072,0.472246,0.224374,...,0.774964,0.925404,0.635816,0.961835,0.971032,0.847891,0.595177,0.145362,0.510444,0.898962
75%,7482.000000,0.914996,0.685232,0.038720,0.781834,0.862377,0.927504,0.599702,0.530289,0.323585,...,0.853582,0.944194,0.665119,0.972649,0.981229,0.877225,0.837599,0.196282,0.761122,0.915091
max,9640.000000,0.975389,0.892780,0.587303,0.942257,0.992752,0.977484,0.711050,0.740644,0.658976,...,0.958374,0.981211,0.762098,0.989709,0.995275,0.944861,0.985489,0.419366,0.990099,0.966875
